In [30]:
#Calendar Webscraping & Google API
#Allen Lau

In [31]:
#importing relevant libraries 
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from IPython.display import HTML
import pendulum as dt
from apiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow

In [32]:
#GET request for the CCNY Calendar
req = requests.get('https://www.ccny.cuny.edu/registrar/fall')
#creating BeautifulSoup Object that holds the parsed HTML content of the website
soup = BeautifulSoup(req.text, 'html.parser')
print(soup.prettify()[:1000])

<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   (window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"NRJS-83581f4fb59b51cbeb7",applicationID:"254978615"};/*! For license information please see nr-loader-rum-1220.min.js.LICENSE.txt */
!function(t,e){"object"==typeof exports&&"object"==typeof module?module.exports=e():"function"==typeof define&&define.amd?define([],e):"object"==typeof exports?exports.NRBA=e():t.NRBA=e()}(self,(function(){return function(){var t,e,n={9034:function(t,e,n)

In [33]:
#finding all tables on the website
all_tables = soup.find_all('table')
print(len(all_tables))
table = all_tables[0]

1


In [34]:
#display first indexed table
HTML(table.prettify())

DATES,DAYS,
August 01,Sunday,Application for degree for January and February 2022 begins
August 18,Wednesday,Last day to apply for Study Abroad
August 24,Tuesday,Last day of Registration; Last day to file ePermit for the Fall 2021; Last day to drop classes for 100% tuition refund;
August 25,Wednesday,Start of Fall Term; Classes begin; Initial Registration Appeals begin;
August 25 - 31,Wednesday - Tuesday,Change of program period; late fees apply
August 26,Thursday,Last day for Independent Study
August 28,Saturday,First day of Saturday Classes
August 31,Tuesday,Last day to add a class to an existing enrollment; Last day for 75% tuition refund; Financial Aid Certification Enrollment Status date; Last day to apply for Audit option; Last day to drop without the grade of 'WD'; Initial Registration Appeals end;
September 01,Wednesday,Verification of Enrollment rosters available to faculty; Course Withdrawal drop period begins (A grade of 'WD' is assigned to students who officially drop a class);
September 03 - 08,Friday - Wednesday,No classes scheduled


In [35]:
#finding the table headers using tag 'th'
header = [row for row in table.find_all('th')]
print(header)

[<th class="text-align-left" scope="col" style="width: 305px;">DATES</th>, <th class="text-align-left" scope="col" style="width: 327px;">DAYS</th>, <th scope="col" style="width: 617px;"> </th>]


In [36]:
#retrieving only the text from the header
col_header = [col.get_text() for col in header]
print(col_header)

['DATES', 'DAYS', '\xa0']


In [37]:
#Renaming Column names
col_header[1] = 'DAYS OF THE WEEK'
col_header[2] = 'TEXT'
print(col_header)

['DATES', 'DAYS OF THE WEEK', 'TEXT']


In [38]:
#display table column headers 
rawdata = [row for row in table.find_all('tr')]
HTML(rawdata[1].prettify())

August 01,Sunday,Application for degree for January and February 2022 begins


In [39]:
#cleaning up rawdata to remove special characters
#created a list of lists by looping through raw data, finding td tags, and getting the text
data = [[col.get_text(strip = True) for col in row.find_all('td')] for row in rawdata[1:]]
print(data[:5])

[['August 01', 'Sunday', 'Application for degree for January and February 2022 begins'], ['August 18', 'Wednesday', 'Last day to apply for Study Abroad'], ['August 24', 'Tuesday', 'Last day of Registration;Last day to file ePermit for the Fall 2021;Last day to drop classes for 100% tuition refund;'], ['August 25', 'Wednesday', 'Start of Fall Term;Classes begin;Initial Registration Appeals begin;'], ['August 25 - 31', 'Wednesday - Tuesday', 'Change of program period; late fees apply']]


In [40]:
#create np array
np_value_rows = np.array(data)
np_value_rows[:4]

array([['August 01', 'Sunday',
        'Application for degree for January and February 2022 begins'],
       ['August 18', 'Wednesday', 'Last day to apply for Study Abroad'],
       ['August 24', 'Tuesday',
        'Last day of Registration;Last day to file ePermit for the Fall 2021;Last day to drop classes for 100% tuition refund;'],
       ['August 25', 'Wednesday',
        'Start of Fall Term;Classes begin;Initial Registration Appeals begin;']],
      dtype='<U274')

In [41]:
#take data (list of lists) and restructure it using *, zip, and list so that it can be converted into a pandas datatframe
series_list = list(zip(*data))
print(series_list)

[('August 01', 'August 18', 'August 24', 'August 25', 'August 25 - 31', 'August 26', 'August 28', 'August 31', 'September 01', 'September 03 - 08', 'September 06', 'September 09', 'September 14', 'September 15', 'September 15 - 16', 'September 23', 'September 24', 'October 01', 'October 08', 'October 11', 'November 01', 'November 02', 'November 04', 'November 06', 'November 23', 'November 25 - 28', 'December 11', 'December 13', 'December 14', 'December 15 - 21', 'December 21', 'December 24 - 25', 'December 27', 'December 28', 'December 31', 'January 1, 2022'), ('Sunday', 'Wednesday', 'Tuesday', 'Wednesday', 'Wednesday - Tuesday', 'Thursday', 'Saturday', 'Tuesday', 'Wednesday', 'Friday - Wednesday', 'Monday', 'Thursday', 'Tuesday', 'Wednesday', 'Wednesday - Thursday', 'Thursday', 'Friday', 'Friday', 'Friday', 'Monday', 'Monday', 'Tuesday', 'Thursday', 'Saturday', 'Tuesday', 'Thursday - Sunday', 'Saturday', 'Monday', 'Tuesday', 'Wednesday - Tuesday', 'Tuesday', 'Friday - Saturday', 'Mond

In [42]:
#create pandas DataFrame 
df = pd.DataFrame(dict(zip(col_header,series_list)))
df.head()

,DATES,DAYS OF THE WEEK,TEXT
0,August 01,Sunday,Application for degree for January and Februar...
1,August 18,Wednesday,Last day to apply for Study Abroad
2,August 24,Tuesday,Last day of Registration;Last day to file ePer...
3,August 25,Wednesday,Start of Fall Term;Classes begin;Initial Regis...
4,August 25 - 31,Wednesday - Tuesday,Change of program period; late fees apply


In [45]:
#setting index as DATES
df1 = df.set_index('DATES')
df1.head()

,DAYS OF THE WEEK,TEXT
DATES,,
August 01,Sunday,Application for degree for January and Februar...
August 18,Wednesday,Last day to apply for Study Abroad
August 24,Tuesday,Last day of Registration;Last day to file ePer...
August 25,Wednesday,Start of Fall Term;Classes begin;Initial Regis...
August 25 - 31,Wednesday - Tuesday,Change of program period; late fees apply


In [46]:
#using the quick way of creating Dataframe Table
df2 = pd.read_html(str(table))[0]
df2 = df2.rename(columns = {'DAYS':'DAYS OF THE WEEK','Unnamed: 2':'TEXT'})
df2 = df2.set_index('DATES')
df2.head()

,DAYS OF THE WEEK,TEXT
DATES,,
August 01,Sunday,Application for degree for January and Februar...
August 18,Wednesday,Last day to apply for Study Abroad
August 24,Tuesday,Last day of Registration; Last day to file eP...
August 25,Wednesday,Start of Fall Term; Classes begin; Initial R...
August 25 - 31,Wednesday - Tuesday,Change of program period; late fees apply


In [48]:
#using pendulum to format dates correctly for Google API
daterange = []

df3 = pd.read_html(str(table))[0]
df3 = df3.rename(columns = {'DAYS':'DAYS OF THE WEEK','Unnamed: 2':'TEXT'})

for index, date in df3['DATES'].items():
    if ',' in date:
        df3.at[index,'DATES'] = dt.parse(date,tz = 'US/Eastern',strict=False).to_date_string()
    elif '-' not in date:
        df3.at[index,'DATES'] = dt.parse(date + ' 2021',tz = 'US/Eastern',strict=False).to_date_string()
    else:
        date_string = date.split(' -')
        date_string[0] = date_string[0] + ' 2021'
        date_string[1] = date_string[0].split(' ')[0] + date_string[1] + ' 2021'
        daterange = ([dt.parse(date_string[0],strict=False,tz = 'US/Eastern').to_date_string(),dt.parse(date_string[1],strict=False,tz = 'US/Eastern').to_date_string()])
        df3.at[index,"DATES"] = daterange

df3 = df3.set_index("DATES")
df3.head()

,DAYS OF THE WEEK,TEXT
DATES,,
2021-08-01,Sunday,Application for degree for January and Februar...
2021-08-18,Wednesday,Last day to apply for Study Abroad
2021-08-24,Tuesday,Last day of Registration; Last day to file eP...
2021-08-25,Wednesday,Start of Fall Term; Classes begin; Initial R...
"[2021-08-25, 2021-08-31]",Wednesday - Tuesday,Change of program period; late fees apply


### 

In [49]:
#setting up authentication 
scopes = ['https://www.googleapis.com/auth/calendar']
flow = InstalledAppFlow.from_client_secrets_file('credentials2.json',scopes=scopes)

In [26]:
#credential input
credentials = flow.run_console()

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=661183248704-mlf7svvf6p20k97qo794qtgnghqf150p.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=16nvX6ugnK2NQ4PJnNAJMQe7AQooEP&prompt=consent&access_type=offline
Enter the authorization code: 4/1AWgavdeDRpsjA6yWwGEwTr_2y0cgi8aRjzrA0wxKEfCXqJbYMHWpms2AKbE


In [27]:
#building service for API  
service = build('calendar','v3',credentials=credentials)

In [29]:
#building request body and using google api to create events 
for index,descr in df3['TEXT'].items():
    if type(index) == str:
        event = {'start':{'date':index},'end':{'date':index},'summary':descr}
        service.events().insert(calendarId='primary',body=event).execute()
    else:
        event = {'start':{'date':index[0]},'end':{'date':index[1]},'summary':descr}
        service.events().insert(calendarId='primary',body=event).execute()